This notebook touches upon: 

* Making a prediction for every pixel
* Working with image segmentation
* Enlarging images with transposed convolutions
* Using bounding boxes for object detection with Faster R-CNN
* Filtering results to reduce false positives

U-Net that has become a de facto design approach for image segmentation.

Image segmentation -> Classification problem, but instead of classifying the whole image, we classify every pixel. So a 200 × 200 image will have 200 × 200 = 40,000 classifications.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision 
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision import transforms


from torch.utils.data import Dataset, DataLoader

from tqdm.autonotebook import tqdm



import os
from imageio import imread
from glob import glob
import json

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import pandas as pd

from sklearn.metrics import accuracy_score

import time

from idlmam import set_seed
from idlmam import train_network, Flatten, View, weight_reset, moveTo